In [1]:
import pandas as pd
import numpy as np ,numpy.random
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
import random
from catboost import Pool, CatBoostRegressor, cv,CatBoostClassifier

# Table Rendements

In [2]:
url = "C:/Users/louis/Documents/Projet Finance Quantitative/"

In [3]:
database = pd.read_excel(url + "Universe_ESG.xlsx")
database = database.T.iloc[1:,]
database = database.reset_index()
database.columns = database.iloc[0,]
database = database.iloc[1:,]
database = database.rename(columns={"Name" : "Date"})
database["Date"] = pd.to_datetime(database["Date"])
Convertir = list(database.columns[1:])
database[Convertir] = database[Convertir].astype(float)
print(database.shape)
database.head(3)
pd.set_option('display.max_rows', 20)
db_rendements = database.copy()
db_rendements=db_rendements.set_index("Date")
db_rendements = db_rendements.stack()
db_rendements = pd.DataFrame(db_rendements,columns = {"Return"})
db_rendements = db_rendements.reset_index()
db_rendements = db_rendements.rename(columns = {0:"Titres"})
lag_1mois = db_rendements.sort_values(["Titres","Date"]).groupby("Titres")["Return"].shift(1)
lag_2mois = db_rendements.sort_values(["Titres","Date"]).groupby("Titres")["Return"].shift(2)
lag_3mois = db_rendements.sort_values(["Titres","Date"]).groupby("Titres")["Return"].shift(3)
lag_4mois = db_rendements.sort_values(["Titres","Date"]).groupby("Titres")["Return"].shift(4)
db_rendements["lag1"]=lag_1mois
db_rendements["lag2"]=lag_2mois
db_rendements["lag3"]=lag_3mois
db_rendements["lag4"]=lag_4mois
db_rendements.sort_values(["Titres","Date"])
db_rendements.iloc[400:405,:]

(119, 51)


,Date,Titres,Return,lag1,lag2,lag3,lag4
400,2010-09-30,HASBRO INC,10.282460,-4.246738,3.167670,2.366127,4.666319
401,2010-09-30,DUKE ENERGY CORP,3.084983,1.906446,6.875000,0.250627,-3.502466
402,2010-09-30,EASTMAN CHEMICAL CO,21.034850,-1.740102,17.391300,-10.988900,-9.757920
403,2010-09-30,BANK OF NEW YORK MELLON CORP,7.708161,-3.230953,1.900434,-9.227942,-12.624480
404,2010-09-30,DOMTAR CORP,8.015224,2.598291,19.023400,-19.343380,-13.608130


##### Valeurs manquantes sur les rendements ?

In [4]:
db_rendements.isna().sum()

Date        0
Titres      0
Return      0
lag1       50
lag2      100
lag3      150
lag4      200
dtype: int64

On peut observer qu'il n'y a pas de valeurs manquantes sur les données de rendement fournies initialement  

In [5]:
db_rendements.to_csv("db_rendements.csv", header=True,index=False)

In [6]:
database.to_csv("database.csv", header=True,index=False)

# Table de jointure

In [7]:
pd.set_option("display.max_rows",500)
Equivalence = pd.read_excel(url + "Universe_ESG.xlsx",usecols=["Ticker","Name"])
Equivalence['Abréviation'] = Equivalence['Ticker'].astype(str).str[:-7]
#Equivalence["Domaine"] = 
Equivalence

,Ticker,Name,Abréviation
0,HES US Equity,HESS CORP,HES US
1,NEM US Equity,NEWMONT GOLDCORP CORP,NEM US
2,INTC US Equity,INTEL CORP,INTC US
3,IFF US Equity,INTL FLAVORS & FRAGRANCES,IFF US
4,MOS US Equity,MOSAIC CO/THE,MOS US
5,EXC US Equity,EXELON CORP,EXC US
6,KMB US Equity,KIMBERLY-CLARK CORP,KMB US
7,JNJ US Equity,JOHNSON & JOHNSON,JNJ US
8,BAX US Equity,BAXTER INTERNATIONAL INC,BAX US
9,FO US Equity,FORD MOTOR CO,FO US


# Table Indicateurs

In [8]:
IND1 = pd.read_excel(url + "Excel_fq_clean.xlsx",sheet_name="EBITDA_TO_REVENUE2")
IND1.rename(columns={"F US Equity" : "FO US Equity","T US Equity" : "AT& US Equity","C US Equity" : "CI US Equity" },inplace=True)
IND2 = pd.read_excel(url + "Excel_fq_clean.xlsx",sheet_name="CURRENT_EV_TO_T12M_EBITDA2")
IND2.rename(columns={"F US Equity" : "FO US Equity","T US Equity" : "AT& US Equity","C US Equity" : "CI US Equity" },inplace=True)
IND3 = pd.read_excel(url + "Excel_fq_clean.xlsx",sheet_name="EBITDA_to_Interest_Expense2")
IND3.rename(columns={"F US Equity" : "FO US Equity","T US Equity" : "AT& US Equity","C US Equity" : "CI US Equity" },inplace=True)
IND4 = pd.read_excel(url + "Excel_fq_clean.xlsx",sheet_name="TOT_DEBT_TO_EBITDA2")
IND4.rename(columns={"F US Equity" : "FO US Equity","T US Equity" : "AT& US Equity","C US Equity" : "CI US Equity" },inplace=True)
IND5 = pd.read_excel(url + "Excel_fq_clean.xlsx",sheet_name="BEST_EST_PE_NXT_YR2")
IND5.loc[0:116,:].fillna(int(-999),inplace=True)
IND5.rename(columns={"F US Equity" : "FO US Equity","T US Equity" : "AT& US Equity","C US Equity" : "CI US Equity" },inplace=True)
IND6 = pd.read_excel(url + "Excel_fq_clean.xlsx",sheet_name="PE_RATIO2")
IND6.rename(columns={"F US Equity" : "FO US Equity","T US Equity" : "AT& US Equity","C US Equity" : "CI US Equity" },inplace=True)
IND7 = pd.read_excel(url + "Excel_fq_clean.xlsx",sheet_name="DIVIDEND_INDICATED_YIELD2")
IND7.rename(columns={"F US Equity" : "FO US Equity","T US Equity" : "AT& US Equity","C US Equity" : "CI US Equity" },inplace=True)
IND8 = pd.read_excel(url + "Excel_fq_clean.xlsx",sheet_name="EQY_DVD_YLD_IND2")
IND8.rename(columns={"F US Equity" : "FO US Equity","T US Equity" : "AT& US Equity","C US Equity" : "CI US Equity" },inplace=True)
IND9 = pd.read_excel(url + "Excel_fq_clean.xlsx",sheet_name="EPS_GROWTH2")
IND9.rename(columns={"F US Equity" : "FO US Equity","T US Equity" : "AT& US Equity","C US Equity" : "CI US Equity" },inplace=True)
indicateurs = IND1.add_suffix('_EBITDA_TO_REVENUE').merge(IND2.add_suffix('_CURRENT_EV_TO_T12M_EBITDA'),how = "left",left_on="Fonds_EBITDA_TO_REVENUE",right_on="Fonds_CURRENT_EV_TO_T12M_EBITDA")
del indicateurs["Fonds_CURRENT_EV_TO_T12M_EBITDA"]
indicateurs = indicateurs.merge(IND3.add_suffix('_EBITDA_to_Interest_Expense'),how = "left",left_on="Fonds_EBITDA_TO_REVENUE",right_on="Fonds_EBITDA_to_Interest_Expense")
del indicateurs["Fonds_EBITDA_to_Interest_Expense"]
indicateurs = indicateurs.merge(IND4.add_suffix('_TOT_DEBT_TO_EBITDA'),how = "left",left_on="Fonds_EBITDA_TO_REVENUE",right_on="Fonds_TOT_DEBT_TO_EBITDA")
del indicateurs["Fonds_TOT_DEBT_TO_EBITDA"]
indicateurs = indicateurs.merge(IND5.add_suffix('_BEST_EST_PE_NXT_YR2'),how = "left",left_on="Fonds_EBITDA_TO_REVENUE",right_on="Fonds_BEST_EST_PE_NXT_YR2")
del indicateurs["Fonds_BEST_EST_PE_NXT_YR2"]
indicateurs = indicateurs.merge(IND6.add_suffix('_PE_RATIO'),how = "left",left_on="Fonds_EBITDA_TO_REVENUE",right_on="Fonds_PE_RATIO")
del indicateurs["Fonds_PE_RATIO"]
indicateurs = indicateurs.merge(IND7.add_suffix('_DIVIDEND_INDICATED_YIELD'),how = "left",left_on="Fonds_EBITDA_TO_REVENUE",right_on="Fonds_DIVIDEND_INDICATED_YIELD")
del indicateurs["Fonds_DIVIDEND_INDICATED_YIELD"]
indicateurs = indicateurs.merge(IND8.add_suffix('_EQY_DVD_YLD_IND'),how = "left",left_on="Fonds_EBITDA_TO_REVENUE",right_on="Fonds_EQY_DVD_YLD_IND")
del indicateurs["Fonds_EQY_DVD_YLD_IND"]
indicateurs = indicateurs.merge(IND9.add_suffix('_EPS_GROWTH'),how = "left",left_on="Fonds_EBITDA_TO_REVENUE",right_on="Fonds_EPS_GROWTH")
del indicateurs["Fonds_EPS_GROWTH"]
indicateurs = indicateurs.rename(columns={"Fonds_EBITDA_TO_REVENUE" : "Dates"})
indicateurs.head(2)

C:\Users\louis\Anaconda3\lib\site-packages\pandas\core\frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


,Dates,HES US Equity_EBITDA_TO_REVENUE,NEM US Equity_EBITDA_TO_REVENUE,INTC US Equity_EBITDA_TO_REVENUE,IFF US Equity_EBITDA_TO_REVENUE,MOS US Equity_EBITDA_TO_REVENUE,EXC US Equity_EBITDA_TO_REVENUE,KMB US Equity_EBITDA_TO_REVENUE,JNJ US Equity_EBITDA_TO_REVENUE,BAX US Equity_EBITDA_TO_REVENUE,...,IBM US Equity_EPS_GROWTH,WELL US Equity_EPS_GROWTH,IR US Equity_EPS_GROWTH,BKR US Equity_EPS_GROWTH,WEC US Equity_EPS_GROWTH,OXY US Equity_EPS_GROWTH,WMB US Equity_EPS_GROWTH,UNP US Equity_EPS_GROWTH,WM US Equity_EPS_GROWTH,CCL US Equity_EPS_GROWTH
0,2010-01-31,5.871767,16.904533,19.417867,6.3521,11.826033,15.841,5.9152,11.730967,3.405067,...,5.653033,-21.9363,NaN,NaN,-2.754833,66.666667,12.6984,13.8889,6.4516,-1.9608
1,2010-02-28,5.871767,16.904533,19.417867,6.3521,11.826033,15.841,5.9152,11.730967,3.405067,...,5.653033,-21.9363,NaN,NaN,-2.754833,66.666667,12.6984,13.8889,6.4516,-1.9608


On observe que chaqu'un des indicateurs possède des variables manquantes. Pour aller même plus loin, chaque titre a des valeurs manquantes pour tous ces indicateurs. Cependant, il sera à noter qu'aucun indicateur n'est disponible pour les deux derniers mois de l'étude (Octobre et Novemebre 2019) par conséquent ces valeurs manquantes ne seront pas prises en compte. 
On a cependant beaucoup d'autres valeurs manquantes : 

In [10]:
Valeurs_manquantes = pd.DataFrame(indicateurs.loc[0:116,:].isna().sum(),columns=["NA"]).reset_index()
Valeurs_manquantes = Valeurs_manquantes[Valeurs_manquantes["NA"]!= 0].reset_index(drop=True)
Valeurs_manquantes["Nom"]=Valeurs_manquantes["index"].astype(str).str[0:3]
Valeurs_manquantes

,index,NA,Nom
0,INTC US Equity_EBITDA_TO_REVENUE,3,INT
1,MOS US Equity_EBITDA_TO_REVENUE,9,MOS
2,CC US Equity_EBITDA_TO_REVENUE,48,CC
3,WFC US Equity_EBITDA_TO_REVENUE,117,WFC
4,HAS US Equity_EBITDA_TO_REVENUE,3,HAS
5,BK US Equity_EBITDA_TO_REVENUE,117,BK
6,AMD US Equity_EBITDA_TO_REVENUE,3,AMD
7,CMA US Equity_EBITDA_TO_REVENUE,117,CMA
8,BKR US Equity_EBITDA_TO_REVENUE,72,BKR
9,CC US Equity_CURRENT_EV_TO_T12M_EBITDA,63,CC


On remarque emaintenant qu'après avoir enlevé les deux denirères périodes,  nous n'avons plus que des valeurs manquantes de longues durées ou complètes (sur toutes les périodes soit 119). 
Nous allons procéder en 2 façons. 
Nous rappellons le but est de remplacer ou non les valeurs manquantes en n'atténuant pas l'effet des modalité présentes pour les indicateurs. On se dit par exmple qu'ici, remplacder par la moyenne ou médiane pourrait être intéressant, cependant nous craignions que cela affecte les tendances réelles des indicateurs. 

In [11]:
Valeurs_manquantes[Valeurs_manquantes["NA"]!=117]

,index,NA,Nom
0,INTC US Equity_EBITDA_TO_REVENUE,3,INT
1,MOS US Equity_EBITDA_TO_REVENUE,9,MOS
2,CC US Equity_EBITDA_TO_REVENUE,48,CC
4,HAS US Equity_EBITDA_TO_REVENUE,3,HAS
6,AMD US Equity_EBITDA_TO_REVENUE,3,AMD
8,BKR US Equity_EBITDA_TO_REVENUE,72,BKR
9,CC US Equity_CURRENT_EV_TO_T12M_EBITDA,63,CC
11,GM US Equity_CURRENT_EV_TO_T12M_EBITDA,9,GM
12,CMI US Equity_CURRENT_EV_TO_T12M_EBITDA,6,CMI
15,BKR US Equity_CURRENT_EV_TO_T12M_EBITDA,90,BKR


In [12]:
Valeurs_manquantes[Valeurs_manquantes["NA"]!=117]["NA"].sum()

1389

In [13]:
Valeurs_manquantes[Valeurs_manquantes["NA"]!=117]["Nom"].value_counts()

CC     8
GM     6
BKR    6
AMD    4
MOS    4
INT    4
HAS    4
UFS    3
CI     3
FO     3
CMI    2
IR     1
CMA    1
HES    1
WY     1
KIM    1
OXY    1
BK     1
Name: Nom, dtype: int64

Donc pour conserver l'effet des valeurs d'indicateurs déjà présentes, nous décidons de ne pas remplacer les valeurs manquantes. Nous proposons à la place d'établir une variable appelée "date_active" qui nous permettra de déterminer la date pour laquelle le titre aura tous ces indicateurs disponibles. C'est à partir de cette date ensuite que nous pourrons commencer à l'utiliser afin de prédire les rendements. Nous mettrons aussi une condition supplémentaire en plus: chacun de ces titres ne peut rentrer dans la modélisation qu' un an après sa date active (cela nous permet de posséder au moins 4 périodes d'indicateurs).

In [14]:
Valeurs_manquantes[Valeurs_manquantes["NA"]==117]

,index,NA,Nom
3,WFC US Equity_EBITDA_TO_REVENUE,117,WFC
5,BK US Equity_EBITDA_TO_REVENUE,117,BK
7,CMA US Equity_EBITDA_TO_REVENUE,117,CMA
10,WFC US Equity_CURRENT_EV_TO_T12M_EBITDA,117,WFC
13,BK US Equity_CURRENT_EV_TO_T12M_EBITDA,117,BK
14,CMA US Equity_CURRENT_EV_TO_T12M_EBITDA,117,CMA
19,WFC US Equity_EBITDA_to_Interest_Expense,117,WFC
21,BK US Equity_EBITDA_to_Interest_Expense,117,BK
22,MO US Equity_EBITDA_to_Interest_Expense,117,MO
23,JLL US Equity_EBITDA_to_Interest_Expense,117,JLL


In [15]:
Valeurs_manquantes[Valeurs_manquantes["NA"]==117]["Nom"].value_counts()

WFC    4
CMA    4
BK     4
AMD    2
MO     1
JLL    1
Name: Nom, dtype: int64

In [16]:
traitement = list(Valeurs_manquantes[Valeurs_manquantes["NA"]==117]["index"])

Ci-dessus, nous procédons d'une façon différente que précédemnt. Etant donné que la date active ne servirait à rien ici car elle nous amènerait à minima à la fin de l'étude et serait donc inutile et nous priverait de chacun de ces titres ce que nous ne souhaitons pas. Nous allons donc pour chacune de ces variables, remplacer les valeurs manquantes par -999. Cela nous permettra donc tous de même d'utiliser ces titres même si leurs données ne seront parfaites. Nous observons que les 3 titres CMA, WFC  et BK ont 4 indicateurs non disponibles soit près de 50% ce qui semble beaucoup. Nous les remplacons pour le moment mais nous réservons le droit de les enlever par la suite.

In [17]:
indicateurs.loc[0:116,:][traitement] = indicateurs.loc[0:116,:][traitement].fillna(value=-999)

C:\Users\louis\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [18]:
pos=0
Ticker_abr=[]
Dates = []
EBITDA_TO_REVENUE = []
CURRENT_EV_TO_T12M_EBITDA = []
EBITDA_to_Interest_Expense=[]
TOT_DEBT_TO_EBITDA =[]
BEST_EST_PE_NXT_YR2 = []
PE_RATIO = []
DIVIDEND_INDICATED_YIELD = []
EQY_DVD_YLD_IND = []
EPS_GROWTH = []
while pos < 50 : 
        a=[x for x in list(indicateurs.columns) if Equivalence['Abréviation'][pos] in x.upper()]
        a.append("Dates")
        table = indicateurs[a]
        #x=Equivalence['Abréviation'][pos]
        x = [Equivalence['Abréviation'][pos]] * len(table)
        Ticker_abr.extend(x)
        Dates.extend(pd.to_datetime(table.iloc[:,9]))
        #Dates.extend(table.iloc[:,8].astype(str))
        EBITDA_TO_REVENUE.extend(table.iloc[:,0])#.shift(1) #comme annonce de marché 
        CURRENT_EV_TO_T12M_EBITDA.extend(table.iloc[:,1])#.shift(1)
        EBITDA_to_Interest_Expense.extend(table.iloc[:,2])#.shift(1)
        TOT_DEBT_TO_EBITDA.extend(table.iloc[:,3])#.shift(1)
        BEST_EST_PE_NXT_YR2.extend(table.iloc[:,4])
        PE_RATIO.extend(table.iloc[:,5])#.shift(1)
        DIVIDEND_INDICATED_YIELD.extend(table.iloc[:,6])#.shift(1)
        EQY_DVD_YLD_IND.extend(table.iloc[:,7])#.shift(1)
        EPS_GROWTH.extend(table.iloc[:,8])#.shift(1)
        pos+=1
Indicateur_titres = pd.DataFrame(list(zip(Ticker_abr,Dates,EBITDA_TO_REVENUE,CURRENT_EV_TO_T12M_EBITDA,EBITDA_to_Interest_Expense,TOT_DEBT_TO_EBITDA,BEST_EST_PE_NXT_YR2,PE_RATIO,DIVIDEND_INDICATED_YIELD,EQY_DVD_YLD_IND,EPS_GROWTH)),columns=["Ticker_abr","Dates","EBITDA_TO_REVENUE","CURRENT_EV_TO_T12M_EBITDA","EBITDA_to_Interest_Expense","TOT_DEBT_TO_EBITDA","BEST_EST_PE_NXT_YR2","PE_RATIO","DIVIDEND_INDICATED_YIELD","EQY_DVD_YLD_IND","EPS_GROWTH"])

Nous avons crée ici une table suivant le même format que la table des rendements créée précédement. Nous avons donc inscrit chaque indicateur en fonction du titre et de son mois. 
Cette façon de procéder nous permettra de venir plugger ces indicateurs à notre table des rendements. Aussi, nous avons mis tous ces indicateurs au mois et non plus au trimestre par souci pratique.

In [19]:
Indicateur_titres.to_csv("db_indicateurs.csv", header=True,index=False)

In [20]:
Indicateur_titres

,Ticker_abr,Dates,EBITDA_TO_REVENUE,CURRENT_EV_TO_T12M_EBITDA,EBITDA_to_Interest_Expense,TOT_DEBT_TO_EBITDA,BEST_EST_PE_NXT_YR2,PE_RATIO,DIVIDEND_INDICATED_YIELD,EQY_DVD_YLD_IND,EPS_GROWTH
0,HES US,2010-01-31,5.871767,1.529033,6.472233,0.2808,-999.000,6.019033,0.213167,0.213167,NaN
1,HES US,2010-02-28,5.871767,1.529033,6.472233,0.2808,-999.000,6.019033,0.213167,0.213167,NaN
2,HES US,2010-03-31,5.871767,1.529033,6.472233,0.2808,-999.000,6.019033,0.213167,0.213167,NaN
3,HES US,2010-04-30,5.690633,1.170033,5.301200,0.2575,-999.000,3.655333,0.264867,0.264867,90.322567
4,HES US,2010-05-31,5.690633,1.170033,5.301200,0.2575,-999.000,3.655333,0.264867,0.264867,90.322567
...,...,...,...,...,...,...,...,...,...,...,...
5945,CCL US,2019-07-31,7.236967,2.396533,7.061233,0.7053,2.962,3.247033,1.525200,1.525200,-4.225367
5946,CCL US,2019-08-31,7.236967,2.396533,7.061233,0.7053,2.962,3.247033,1.525200,1.525200,-4.225367
5947,CCL US,2019-09-30,7.236967,2.396533,7.061233,0.7053,2.962,3.247033,1.525200,1.525200,-4.225367
5948,CCL US,2019-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
Indicateur_titres.isna().sum()

Ticker_abr                      0
Dates                           0
EBITDA_TO_REVENUE             238
CURRENT_EV_TO_T12M_EBITDA     268
EBITDA_to_Interest_Expense    283
TOT_DEBT_TO_EBITDA            274
BEST_EST_PE_NXT_YR2           100
PE_RATIO                      310
DIVIDEND_INDICATED_YIELD      247
EQY_DVD_YLD_IND               247
EPS_GROWTH                    322
dtype: int64

# Table Activité

In [22]:
def départ(df):
    titre = []
    début = []
    for company in list(df.columns[1:]):
        ent = company
        start = df[["Fonds","{}".format(ent)]].dropna()["Fonds"].min()
        if start > pd.to_datetime('2010-01-31 00:00:00') : 
            titre.append(company)
            début.append(start)
        départ_différé = pd.DataFrame(list(zip(titre,début)),columns=["titre","start"])
    return départ_différé
fusion =départ(IND1)
fusion

,titre,start
0,CC US Equity,2013-04-30
1,BKR US Equity,2016-01-31


In [23]:
fusion =départ(IND1)
for tb in [IND2,IND3,IND4,IND6,IND7,IND8,IND9]:  #IND5
    tb2 = départ(tb)
    fusion = fusion.merge(tb2,how = "outer",left_on="titre",right_on="titre")
fusion.columns=["Tickers","EBITDA_TO_REVENUE","CURRENT_EV_TO_T12M_EBITDA","EBITDA_to_Interest_Expense","TOT_DEBT_TO_EBITDA","PE_RATIO","DIVIDEND_INDICATED_YIELD","EQY_DVD_YLD_IND","EPS_GROWTH"]

fusion["Date_active"] = fusion.loc[:,"EBITDA_TO_REVENUE":"EPS_GROWTH"].values.max(axis=1)
#"Date_active" La date à laquelle chacun de ces titres rentrera dans notre portefeuille.
fusion.head(4)

,Tickers,EBITDA_TO_REVENUE,CURRENT_EV_TO_T12M_EBITDA,EBITDA_to_Interest_Expense,TOT_DEBT_TO_EBITDA,PE_RATIO,DIVIDEND_INDICATED_YIELD,EQY_DVD_YLD_IND,EPS_GROWTH,Date_active
0,CC US Equity,2013-04-30,2015-04-30,2014-07-31,2014-01-31,2016-04-30,2015-04-30,2015-04-30,2016-01-31,2016-04-30
1,BKR US Equity,2016-01-31,2017-07-31,2016-01-31,2017-07-31,2018-01-31,NaT,NaT,2018-10-31,2018-10-31
2,GM US Equity,NaT,2010-10-31,NaT,2010-04-30,2010-10-31,2014-01-31,2014-01-31,2010-07-31,2014-01-31
3,CMI US Equity,NaT,2010-07-31,NaT,2010-07-31,NaT,NaT,NaT,NaT,2010-07-31


In [24]:
fusion[["Tickers","Date_active"]]

,Tickers,Date_active
0,CC US Equity,2016-04-30
1,BKR US Equity,2018-10-31
2,GM US Equity,2014-01-31
3,CMI US Equity,2010-07-31
4,INTC US Equity,2010-10-31
5,OXY US Equity,2011-10-31
6,WY US Equity,2010-10-31
7,BK US Equity,2010-07-31
8,CI US Equity,2011-01-31
9,CMA US Equity,2010-07-31


In [25]:
fusion.to_csv("Start_indicateurs.csv",header=True,index=False)

# Table d'entraînement 

In [26]:
print(Indicateur_titres.shape) # A vérifier, on devrait normalement avoir le meme nombre d'indicateur que de rendements
# car c'est indexé sur les dates.
print(Equivalence.shape)
print(db_rendements.shape)

(5950, 11)
(50, 3)
(5875, 7)


In [27]:
Train_Dataset = Equivalence.merge(Indicateur_titres,how="left", left_on="Abréviation",right_on="Ticker_abr")
del Train_Dataset["Ticker_abr"]
Train_Dataset = db_rendements.merge(Train_Dataset,how="left", left_on=["Date","Titres"],right_on=["Dates","Name"])
del Train_Dataset["Name"]
del Train_Dataset["Dates"]
del Train_Dataset["Abréviation"]
temp = fusion[["Tickers","Date_active"]]
Train_Dataset = Train_Dataset.merge(temp,how="left",left_on="Ticker",right_on="Tickers")
del Train_Dataset["Tickers"]
Train_Dataset["Date_active"] = Train_Dataset["Date_active"].fillna(pd.to_datetime("1997-04-24"))
Train_Dataset.shape#On rajoute une année à la date  d'entrée en activité du titre pour avoir plusieurs trimestre d'indicateur.

(5875, 18)

In [29]:
# Suppression des titres aux périodes où nous ne les utiliserons pas :
Train_setF = Train_Dataset[Train_Dataset["Date"] > Train_Dataset["Date_active"]]
Train_setF['month'] = Train_setF['Date'].dt.month
Train_setF["période"] = np.where(Train_setF["month"].isin([1,4,7,10]),1,np.where(Train_setF["month"].isin([2,5,8,11]),2,np.where(Train_setF["month"].isin([3,6,9,12]),3,None)))
Train_setF.shape

C:\Users\louis\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\louis\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(5605, 20)

In [30]:
Train_setF

,Date,Titres,Return,lag1,lag2,lag3,lag4,Ticker,EBITDA_TO_REVENUE,CURRENT_EV_TO_T12M_EBITDA,EBITDA_to_Interest_Expense,TOT_DEBT_TO_EBITDA,BEST_EST_PE_NXT_YR2,PE_RATIO,DIVIDEND_INDICATED_YIELD,EQY_DVD_YLD_IND,EPS_GROWTH,Date_active,month,période
1,2010-01-31,NEWMONT GOLDCORP CORP,-9.406045,NaN,NaN,NaN,NaN,NEM US Equity,16.904533,2.190000,4.858967,0.354567,-999.0,4.947800,0.261800,0.261800,59.166667,1997-04-24,1,1
3,2010-01-31,INTL FLAVORS & FRAGRANCES,-3.330092,NaN,NaN,NaN,NaN,IFF US Equity,6.352100,3.541267,3.261400,0.763733,-999.0,5.414900,0.699267,0.699267,11.111100,1997-04-24,1,1
4,2010-01-31,MOSAIC CO/THE,-10.413530,NaN,NaN,NaN,NaN,MOS US Equity,11.826033,6.835100,13.253000,0.261067,-999.0,30.601933,0.109700,0.109700,56.565667,1997-04-24,1,1
5,2010-01-31,EXELON CORP,-6.650297,NaN,NaN,NaN,NaN,EXC US Equity,15.841000,1.718900,3.861567,0.566600,-999.0,3.738633,1.597800,1.597800,1.543200,1997-04-24,1,1
6,2010-01-31,KIMBERLY-CLARK CORP,-6.780725,NaN,NaN,NaN,NaN,KMB US Equity,5.915200,2.934567,4.688533,0.490567,-999.0,4.130667,1.460200,1.460200,-2.040800,1997-04-24,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,2019-11-30,OCCIDENTAL PETROLEUM CORP,-4.765432,-8.927367,4.073635,-15.342680,2.147971,OXY US Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-10-31,11,2
5871,2019-11-30,WILLIAMS COS INC,1.837741,-7.273483,3.544075,-4.220779,-12.125540,WMB US Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-04-24,11,2
5872,2019-11-30,UNION PACIFIC CORP,6.946350,2.148413,0.012349,-9.457417,6.410029,UNP US Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-04-24,11,2
5873,2019-11-30,WASTE MANAGEMENT INC,0.623830,-2.426087,-3.230081,2.008547,1.412846,WM US Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-04-24,11,2


In [30]:
Train_setF.to_csv("db_train.csv",header=True,index=False)

##### FIN 